<a href="https://colab.research.google.com/github/jayjinnie/gbdcon/blob/main/%EA%B2%BD%EB%B6%81_%EA%B5%AC%EB%A7%A4_%EC%9D%B4%EB%A0%A5_%EA%B8%B0%EB%B0%98_%EC%9E%A0%EC%9E%AC_%EC%9A%94%EC%9D%B8_%ED%98%91%EC%97%85_%ED%95%84%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 고객의 구매 이력을 기반으로 맞춤형 상품을 추천하는 시스템

# pivot 테이블 만들기
#### 고객-상품 선호도 데이터를 pivot 테이블로 변환

In [ ]:
import pandas as pd
df_ratings = pd.read_csv("선호도 테이블.csv")
df_ratings

,회원번호,상품코드,선호도
0,1,1586858861,18.680282
1,1,1602651612,27.532331
2,1,1594681077,146.906185
3,1,1601251488,20.818027
4,1,1601251488,20.818027
...,...,...,...
16457,169279,1601971908,27.192747
16458,169280,1601972061,28.672495
16459,169282,1585032127,16.827011
16460,169286,1581252441,15.120096


In [ ]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16462 entries, 0 to 16461
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   회원번호    16462 non-null  int64  
 1   상품코드    16462 non-null  int64  
 2   선호도     16462 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 386.0 KB


In [ ]:
df_merchandise = pd.read_excel("장바구니테이블.xlsx", header=1)
df_merchandise.head() 

,장바구니번호,주문번호,회원아이디,상품코드,상품명,상품 배송비,상품가격,포인트,쿠폰금액,주문옵션,주문수량,옵션가격,업체ID,즉시할인쿠폰,단골할인쿠폰,배송방법,배송일,배송완료예정일,지원가격,배송비,추가배송비
0,2079375,9a8c27650c828207d03dbb34fb5d17dbbd2452b602e829...,309b2e924ff654dedd644f05ecc48525f498aa6abb042d...,1.575014e+09,[풍요로운농장] 감시골 양파즙,0,25000,0,0,[풍요로운농장] 감시골 양파즙,1,0,3be89a2b3a2c380400ce214a5bc42f473f595cfbe6ecca...,0,0,home,2020-01-16 00:00:00,0000-00-00,0,0,0
1,2079381,5f6410d90f5f42b49efeebecee50a989aee88658569f52...,1fd648ccc7c4f02861699b39660e84d3e8a35d25c35189...,1.575014e+09,[풍요로운농장] 감시골 양파즙,0,25000,0,0,[풍요로운농장] 감시골 양파즙,1,0,3be89a2b3a2c380400ce214a5bc42f473f595cfbe6ecca...,0,0,home,0000-00-00,0000-00-00,0,0,0
2,2079385,1291ff3422b02515594f564eae65980860baa03bf55809...,4756031f0b9f62c7c1673a1f9d956005829ab3db0ba9b5...,1.577772e+09,★제503수요특가★안동마(산마) 상품 - 5kg [풍요로운농장],0,33000,0,0,★제503수요특가★안동마(산마) 상품 - 5kg [풍요로운농장],1,0,3be89a2b3a2c380400ce214a5bc42f473f595cfbe6ecca...,3000,0,home,2020-01-02 00:00:00,2020-01-03 00:00:00,0,0,0
3,2079386,1291ff3422b02515594f564eae65980860baa03bf55809...,4756031f0b9f62c7c1673a1f9d956005829ab3db0ba9b5...,1.577147e+09,★제503수요특가★건강하게 키운 사과 9kg42과,0,33000,0,0,★제503수요특가★건강하게 키운 사과 9kg42과,1,0,be5d427f2b9175f2e358b1be25913e8cf966e91bc0374f...,8000,0,home,2020-01-02 00:00:00,2020-01-03 00:00:00,0,0,0
4,2079399,063d48e1e8eb1a865670286467c6ccf8b6a3e62d1d35d3...,df1f4bc181a94f88ca5fa576441d884f14ae4c70f70cc0...,1.575014e+09,[풍요로운농장] 감시골 양파즙,0,25000,0,0,[풍요로운농장] 감시골 양파즙,1,0,3be89a2b3a2c380400ce214a5bc42f473f595cfbe6ecca...,0,0,home,0000-00-00,0000-00-00,0,0,0


In [ ]:
df_merchandise.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80186 entries, 0 to 80185
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   장바구니번호   80186 non-null  int64  
 1   주문번호     80186 non-null  object 
 2   회원아이디    80186 non-null  object 
 3   상품코드     80182 non-null  float64
 4   상품명      80186 non-null  object 
 5   상품 배송비   80186 non-null  int64  
 6   상품가격     80186 non-null  int64  
 7   포인트      80186 non-null  int64  
 8   쿠폰금액     80186 non-null  int64  
 9   주문옵션     73921 non-null  object 
 10  주문수량     80186 non-null  int64  
 11  옵션가격     80186 non-null  int64  
 12  업체ID     80186 non-null  object 
 13  즉시할인쿠폰   80186 non-null  int64  
 14  단골할인쿠폰   80186 non-null  int64  
 15  배송방법     80186 non-null  object 
 16  배송일      80186 non-null  object 
 17  배송완료예정일  80186 non-null  object 
 18  지원가격     80186 non-null  int64  
 19  배송비      80186 non-null  int64  
 20  추가배송비    80186 non-null  int64  
dtypes: float64(1

In [ ]:
df_custom_merc_ratings = df_ratings.pivot_table(
    index='회원번호', 
    columns='상품코드', 
    values='선호도'
).fillna(0)
df_custom_merc_ratings

상품코드,1577870016,1577933738,1577947433,1577948991,1577950662,1577971381,1578032329,1578043497,1578150796,1578151031,1578215787,1578270709,1578391549,1578556644,1578557663,1578558124,1578622332,1578623254,1578650197,1578826303,1578883413,1578883630,1578930953,1578932858,1578933466,1578933657,1578933961,1578934047,1578934213,1578934433,1578934758,1578934859,1578934943,1578935132,1578935254,1578935386,1578960613,1578971894,1578977705,1578977947,...,1607064393,1607064407,1607064501,1607064512,1607064705,1607064798,1607064811,1607064822,1607064833,1607064985,1607064996,1607065142,1607065303,1607065410,1607066126,1607069571,1607069676,1607082898,1607220651,1607221216,1607317799,1607317883,1607318004,1607391075,1607391737,1607419720,1607478386,1607499643,1607948519,1607948747,1607950634,1607981571,1608022246,1608097776,1608101013,1608101688,1608173203,1608187426,1608600164,1608699587
회원번호,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
169280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
169282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# N명의 고객들이 M개의 상품에 매긴 각각의 평균 선호도 구하기
#### 이렇게 구한 평균 선호도 값을 고객-상품 선호도 값에서 빼기

In [ ]:
import numpy as np

# matrix는 pivot table값을 numpy matrix로 만든 것
matrix = df_custom_merc_ratings.values

# customer_ratings_mean은 고객의 평균 선호도
custom_ratings_mean = np.mean(matrix, axis=1)

# R_user_mean: 고객-상품에 대하여 고객 평균 선호도를 뺀 것
matrix_custom_mean = matrix - custom_ratings_mean.reshape(-1,1)

In [ ]:
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
matrix_custom_mean

array([[-0.27359344, -0.27359344, -0.27359344, ..., -0.27359344,
        -0.27359344, -0.27359344],
       [-0.07100404, -0.07100404, -0.07100404, ..., -0.07100404,
        -0.07100404, -0.07100404],
       [-0.08872174, -0.08872174, -0.08872174, ..., -0.08872174,
        -0.08872174, -0.08872174],
       ...,
       [-0.01058303, -0.01058303, -0.01058303, ..., -0.01058303,
        -0.01058303, -0.01058303],
       [-0.00950949, -0.00950949, -0.00950949, ..., -0.00950949,
        -0.00950949, -0.00950949],
       [-0.02009409, -0.02009409, -0.02009409, ..., -0.02009409,
        -0.02009409, -0.02009409]])

In [ ]:
pd.DataFrame(matrix_custom_mean, columns = df_custom_merc_ratings.columns)

상품코드,1577870016,1577933738,1577947433,1577948991,1577950662,1577971381,1578032329,1578043497,1578150796,1578151031,1578215787,1578270709,1578391549,1578556644,1578557663,1578558124,1578622332,1578623254,1578650197,1578826303,1578883413,1578883630,1578930953,1578932858,1578933466,1578933657,1578933961,1578934047,1578934213,1578934433,1578934758,1578934859,1578934943,1578935132,1578935254,1578935386,1578960613,1578971894,1578977705,1578977947,...,1607064393,1607064407,1607064501,1607064512,1607064705,1607064798,1607064811,1607064822,1607064833,1607064985,1607064996,1607065142,1607065303,1607065410,1607066126,1607069571,1607069676,1607082898,1607220651,1607221216,1607317799,1607317883,1607318004,1607391075,1607391737,1607419720,1607478386,1607499643,1607948519,1607948747,1607950634,1607981571,1608022246,1608097776,1608101013,1608101688,1608173203,1608187426,1608600164,1608699587
0,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,...,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593,-0.273593
1,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,...,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004,-0.071004
2,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,...,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722,-0.088722
3,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,...,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.023095,-0.0230

# SVD (Singular Value Decomposition, 특이값 분해)를 이용한 잠재 요인 협업 필터링 (Latent Factor Collaborative Filtering)
#### mxn 크기의 A 행렬을 mxm 크기의 U 행렬, mxn 크기의 sigma k 행렬, nxn 크기의 Vt 행렬로 분해한 뒤, 대각 성분(A 행렬의 특이값) 중 상위 p개를 추출하기
#### 원 행렬로 100% 복원할 수는 없으나 기존의 A 행렬과 거의 근사한 값을 추출할 수 있음

In [ ]:
from scipy.sparse.linalg import svds
from scipy.linalg import svd

U, sigma, Vt = svds(matrix_custom_mean, k=12)
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(8988, 12)
(12,)
(12, 1590)


In [ ]:
sigma = np.diag(sigma)
sigma.shape

(12, 12)

In [ ]:
sigma[0]

array([3080.53931685,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ])

In [ ]:
sigma[1]

array([   0.        , 3625.64183027,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ])

#### U, Sigma, Vt의 내적을 수행, 고객의 선호도 평균을 더하여 원본 행렬로 복구한 뒤, 이 행렬에 고객 평균 선호도를 적용하기

In [ ]:
svd_custom_pred_ratings = np.dot(np.dot(U, sigma), Vt) + custom_ratings_mean.reshape(-1,1)

In [ ]:
df_svd_preds = pd.DataFrame(svd_custom_pred_ratings, columns = df_custom_merc_ratings.columns)
df_svd_preds

상품코드,1577870016,1577933738,1577947433,1577948991,1577950662,1577971381,1578032329,1578043497,1578150796,1578151031,1578215787,1578270709,1578391549,1578556644,1578557663,1578558124,1578622332,1578623254,1578650197,1578826303,1578883413,1578883630,1578930953,1578932858,1578933466,1578933657,1578933961,1578934047,1578934213,1578934433,1578934758,1578934859,1578934943,1578935132,1578935254,1578935386,1578960613,1578971894,1578977705,1578977947,...,1607064393,1607064407,1607064501,1607064512,1607064705,1607064798,1607064811,1607064822,1607064833,1607064985,1607064996,1607065142,1607065303,1607065410,1607066126,1607069571,1607069676,1607082898,1607220651,1607221216,1607317799,1607317883,1607318004,1607391075,1607391737,1607419720,1607478386,1607499643,1607948519,1607948747,1607950634,1607981571,1608022246,1608097776,1608101013,1608101688,1608173203,1608187426,1608600164,1608699587
0,0.277008,0.277001,0.276990,0.277008,0.276682,0.277009,0.276836,0.277008,0.276990,0.277008,0.276967,0.276997,0.275438,0.276703,0.276968,0.272327,0.275520,0.275922,0.276847,0.277008,0.277008,0.277008,0.277007,0.277008,0.277008,0.277017,0.277009,0.277008,0.277008,0.277009,0.277000,0.277008,0.277009,0.277009,0.277008,0.277009,0.276985,0.276989,0.276940,0.276992,...,0.274007,0.276399,0.273799,0.276800,0.276658,0.276695,0.277005,0.276930,0.276708,0.275889,0.274397,0.276346,0.275276,0.276356,0.276754,0.277009,0.277010,0.277008,0.276895,0.276765,0.277008,0.277010,0.277008,0.277007,0.277001,0.277011,0.276979,0.277009,0.277008,0.277008,0.277008,0.276901,0.277009,0.277008,0.276899,0.276688,0.276990,0.275879,0.277007,0.276709
1,0.071255,0.071255,0.071256,0.071255,0.071274,0.071255,0.071258,0.071255,0.071253,0.071255,0.071257,0.071255,0.071318,0.071276,0.071251,0.071424,0.071302,0.071289,0.071267,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071252,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071254,0.071255,...,0.071350,0.071272,0.071369,0.071249,0.071244,0.071228,0.071255,0.071257,0.071266,0.071280,0.071295,0.071253,0.071317,0.071244,0.071274,0.071255,0.071254,0.071255,0.071263,0.071274,0.071253,0.071254,0.071255,0.071255,0.071255,0.071254,0.071255,0.071255,0.071255,0.071255,0.071255,0.071263,0.071255,0.071255,0.071254,0.071262,0.071255,0.071282,0.071255,0.071255
2,0.089616,0.089614,0.089611,0.089616,0.089859,0.089616,0.089554,0.089616,0.089609,0.089616,0.089604,0.089614,0.089333,0.089770,0.089596,0.088724,0.089258,0.089355,0.089559,0.089616,0.089616,0.089616,0.089616,0.089616,0.089616,0.089617,0.089616,0.089616,0.089616,0.089616,0.089613,0.089616,0.089616,0.089616,0.089616,0.089616,0.089608,0.089610,0.089591,0.089610,...,0.088737,0.089726,0.088903,0.089538,0.089736,0.089505,0.089615,0.089588,0.089673,0.089422,0.088656,0.089587,0.089347,0.089379,0.089527,0.089616,0.089616,0.089616,0.089575,0.089528,0.089616,0.089616,0.089616,0.089616,0.089614,0.089617,0.089606,0.089616,0.089616,0.089616,0.089616,0.089687,0.089616,0.089616,0.089575,0.089499,0.089609,0.089204,0.089616,0.089726
3,0.023322,0.023321,0.023321,0.023322,0.023323,0.023322,0.023319,0.023322,0.023321,0.023322,0.023320,0.023321,0.023241,0.023319,0.023318,0.023076,0.023230,0.023255,0.023320,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023321,0.023322,0.023322,0.023322,0.023322,0.023322,0.023320,0.023321,0.023316,0.023320,...,0.023128,0.023297,0.023133,0.023304,0.023311,0.023301,0.023322,0.023316,0.023312,0.023269,0.023130,0.023306,0.023225,0.023268,0.023319,0.023322,0.023322,0.023322,0.023321,0.023320,0.023322,0.023322,0.023322,0.023322,0.023321,0.023322,0.023320,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023312,0.023295,0.023320,0.023229,0.023322,0.023319
4,0.082882,0.082884,0.082888,0.082882,0.083058,0.082882,0.082940,0.082882,0.082884,0.082882,0.082896,0.082886,0.083406,0.083039,0.082882,0.084402,0.083320,0.083202,0.082954,0.082882,0.082882,0.082882,0.08288

#### SVD 특이값 분해를 사용하여 행렬분해를 기반으로 데이터 변형 완료

# 고객 구매 이력을 기반으로 가장 연관성이 높은 상품을 추천하는 함수 만들기
#### 인자로 회원 정보, 상품 정보, 선호도 등을 받음
#### 회원 정보에 SVD로 나온 결과의 상품 선호도가 가장 높은 데이터 순으로 정렬
#### 고객이 구매한 상품을 제외
#### 고객이 구매하지 않은 상품 중 유사한 선호도를 지닌 상품을 추천

In [ ]:
customer_row_number = df_ratings.회원번호

In [ ]:
len(df_svd_preds.reset_index()["index"])

8988

In [ ]:
df_svd_preds.set_index("회원번호",inplace=True)

In [ ]:
df_svd_preds

상품코드,1577870016,1577933738,1577947433,1577948991,1577950662,1577971381,1578032329,1578043497,1578150796,1578151031,1578215787,1578270709,1578391549,1578556644,1578557663,1578558124,1578622332,1578623254,1578650197,1578826303,1578883413,1578883630,1578930953,1578932858,1578933466,1578933657,1578933961,1578934047,1578934213,1578934433,1578934758,1578934859,1578934943,1578935132,1578935254,1578935386,1578960613,1578971894,1578977705,1578977947,...,1607064393,1607064407,1607064501,1607064512,1607064705,1607064798,1607064811,1607064822,1607064833,1607064985,1607064996,1607065142,1607065303,1607065410,1607066126,1607069571,1607069676,1607082898,1607220651,1607221216,1607317799,1607317883,1607318004,1607391075,1607391737,1607419720,1607478386,1607499643,1607948519,1607948747,1607950634,1607981571,1608022246,1608097776,1608101013,1608101688,1608173203,1608187426,1608600164,1608699587
0,0.277008,0.277001,0.276990,0.277008,0.276682,0.277009,0.276836,0.277008,0.276990,0.277008,0.276967,0.276997,0.275438,0.276703,0.276968,0.272327,0.275520,0.275922,0.276847,0.277008,0.277008,0.277008,0.277007,0.277008,0.277008,0.277017,0.277009,0.277008,0.277008,0.277009,0.277000,0.277008,0.277009,0.277009,0.277008,0.277009,0.276985,0.276989,0.276940,0.276992,...,0.274007,0.276399,0.273799,0.276800,0.276658,0.276695,0.277005,0.276930,0.276708,0.275889,0.274397,0.276346,0.275276,0.276356,0.276754,0.277009,0.277010,0.277008,0.276895,0.276765,0.277008,0.277010,0.277008,0.277007,0.277001,0.277011,0.276979,0.277009,0.277008,0.277008,0.277008,0.276901,0.277009,0.277008,0.276899,0.276688,0.276990,0.275879,0.277007,0.276709
1,0.071255,0.071255,0.071256,0.071255,0.071274,0.071255,0.071258,0.071255,0.071253,0.071255,0.071257,0.071255,0.071318,0.071276,0.071251,0.071424,0.071302,0.071289,0.071267,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071252,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071255,0.071254,0.071255,...,0.071350,0.071272,0.071369,0.071249,0.071244,0.071228,0.071255,0.071257,0.071266,0.071280,0.071295,0.071253,0.071317,0.071244,0.071274,0.071255,0.071254,0.071255,0.071263,0.071274,0.071253,0.071254,0.071255,0.071255,0.071255,0.071254,0.071255,0.071255,0.071255,0.071255,0.071255,0.071263,0.071255,0.071255,0.071254,0.071262,0.071255,0.071282,0.071255,0.071255
2,0.089616,0.089614,0.089611,0.089616,0.089859,0.089616,0.089554,0.089616,0.089609,0.089616,0.089604,0.089614,0.089333,0.089770,0.089596,0.088724,0.089258,0.089355,0.089559,0.089616,0.089616,0.089616,0.089616,0.089616,0.089616,0.089617,0.089616,0.089616,0.089616,0.089616,0.089613,0.089616,0.089616,0.089616,0.089616,0.089616,0.089608,0.089610,0.089591,0.089610,...,0.088737,0.089726,0.088903,0.089538,0.089736,0.089505,0.089615,0.089588,0.089673,0.089422,0.088656,0.089587,0.089347,0.089379,0.089527,0.089616,0.089616,0.089616,0.089575,0.089528,0.089616,0.089616,0.089616,0.089616,0.089614,0.089617,0.089606,0.089616,0.089616,0.089616,0.089616,0.089687,0.089616,0.089616,0.089575,0.089499,0.089609,0.089204,0.089616,0.089726
3,0.023322,0.023321,0.023321,0.023322,0.023323,0.023322,0.023319,0.023322,0.023321,0.023322,0.023320,0.023321,0.023241,0.023319,0.023318,0.023076,0.023230,0.023255,0.023320,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023321,0.023322,0.023322,0.023322,0.023322,0.023322,0.023320,0.023321,0.023316,0.023320,...,0.023128,0.023297,0.023133,0.023304,0.023311,0.023301,0.023322,0.023316,0.023312,0.023269,0.023130,0.023306,0.023225,0.023268,0.023319,0.023322,0.023322,0.023322,0.023321,0.023320,0.023322,0.023322,0.023322,0.023322,0.023321,0.023322,0.023320,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023322,0.023312,0.023295,0.023320,0.023229,0.023322,0.023319
4,0.082882,0.082884,0.082888,0.082882,0.083058,0.082882,0.082940,0.082882,0.082884,0.082882,0.082896,0.082886,0.083406,0.083039,0.082882,0.084402,0.083320,0.083202,0.082954,0.082882,0.082882,0.082882,0.08288

In [ ]:
def recommend_merchandise(df_svd_preds, customer_id, ori_merc_df, ori_ratings_df, num_recommendations):

  # 최종적으로 만든 pred_df에서 고객 index에 따라 상품 데이터 정렬하기 -> 상품 선호도가 높은 순으로 정렬됨
  sorted_customer_pred=df_svd_preds.loc[df_svd_preds["회원번호"]==customer_id].set_index("회원번호").transpose().sort_values(by=customer_id,ascending=False).transpose()

  # 원본 선호도 데이터에서 회원번호에 해당하는 데이터 추출
  customer_data = df_ratings[df_ratings["회원번호"]==customer_id]

  # 추출한 회원 데이터와 원본 상품 데이터를 합침
  customer_history = customer_data.merge(df_merchandise, on='상품코드').sort_values(['선호도'], ascending=False)

  # 원본 상품 데이터에서 고객이 구매한 상품을 제외한 데이터를 추출
  recommendations = df_merchandise[~df_merchandise['상품코드'].isin(customer_history['상품코드'])]

  # 고객의 선호도가 높은 순으로 정렬된 데이터와, 위의 recommendation 데이터를 합침
  recommendations = recommendations.merge(pd.DataFrame(sorted_customer_pred.transpose()).reset_index(), on='상품코드')
 
  # 고객의 선호도가 높은 순으로 정렬된 데이터와, 위의 recommendation 데이터를 합침
  recommendations = recommendations.rename(columns={customer_id:"추천 예상"}).sort_values(by="추천 예상",ascending=False).head(num_recommendations)

  return customer_history, recommendations

In [ ]:
pd.DataFrame(sorted_customer_pred.transpose()).reset_index().rename(columns={customer_id:"선호도"}

회원번호,상품코드,60251
0,1592373538,0.100504
1,1595904905,0.096348
2,1589126164,0.084821
3,1588214366,0.079791
4,1593673443,0.075282
...,...,...
1585,1603865613,0.010386
1586,1604385982,0.002753
1587,1584201031,0.002247
1588,1606464566,-0.002416


In [ ]:
df_merchandise[~df_merchandise['상품코드'].isin(customer_history['상품코드'])]

,장바구니번호,주문번호,회원아이디,상품코드,상품명,상품 배송비,상품가격,포인트,쿠폰금액,주문옵션,주문수량,옵션가격,업체ID,즉시할인쿠폰,단골할인쿠폰,배송방법,배송일,배송완료예정일,지원가격,배송비,추가배송비
0,2079375,9a8c27650c828207d03dbb34fb5d17dbbd2452b602e829...,309b2e924ff654dedd644f05ecc48525f498aa6abb042d...,1.575014e+09,[풍요로운농장] 감시골 양파즙,0,25000,0,0,[풍요로운농장] 감시골 양파즙,1,0,3be89a2b3a2c380400ce214a5bc42f473f595cfbe6ecca...,0,0,home,2020-01-16 00:00:00,0000-00-00,0,0,0
1,2079381,5f6410d90f5f42b49efeebecee50a989aee88658569f52...,1fd648ccc7c4f02861699b39660e84d3e8a35d25c35189...,1.575014e+09,[풍요로운농장] 감시골 양파즙,0,25000,0,0,[풍요로운농장] 감시골 양파즙,1,0,3be89a2b3a2c380400ce214a5bc42f473f595cfbe6ecca...,0,0,home,0000-00-00,0000-00-00,0,0,0
2,2079385,1291ff3422b02515594f564eae65980860baa03bf55809...,4756031f0b9f62c7c1673a1f9d956005829ab3db0ba9b5...,1.577772e+09,★제503수요특가★안동마(산마) 상품 - 5kg [풍요로운농장],0,33000,0,0,★제503수요특가★안동마(산마) 상품 - 5kg [풍요로운농장],1,0,3be89a2b3a2c380400ce214a5bc42f473f595cfbe6ecca...,3000,0,home,2020-01-02 00:00:00,2020-01-03 00:00:00,0,0,0
3,2079386,1291ff3422b02515594f564eae65980860baa03bf55809...,4756031f0b9f62c7c1673a1f9d956005829ab3db0ba9b5...,1.577147e+09,★제503수요특가★건강하게 키운 사과 9kg42과,0,33000,0,0,★제503수요특가★건강하게 키운 사과 9kg42과,1,0,be5d427f2b9175f2e358b1be25913e8cf966e91bc0374f...,8000,0,home,2020-01-02 00:00:00,2020-01-03 00:00:00,0,0,0
4,2079399,063d48e1e8eb1a865670286467c6ccf8b6a3e62d1d35d3...,df1f4bc181a94f88ca5fa576441d884f14ae4c70f70cc0...,1.575014e+09,[풍요로운농장] 감시골 양파즙,0,25000,0,0,[풍요로운농장] 감시골 양파즙,1,0,3be89a2b3a2c380400ce214a5bc42f473f595cfbe6ecca...,0,0,home,0000-00-00,0000-00-00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80181,2699790,370c05718f52e6003a13545d4a81d1fce5a6d8aee63f1b...,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,1.570795e+09,[회룡포장수진품] 살짝 볶아 거피해 고소한 어머니들깨가루 250g (국산들깨100%사용),0,10000,0,0,NaN,5,0,b0a4c458a322cf7977abf17bd2d3e97f9203327fb4c291...,0,0,home,2021-01-04 00:00:00,2021-01-05 00:00:00,0,0,0
80182,2699791,e61f1b8ef6c010b68f6d3284a3a4fbb558158fd75b4c40...,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,1.592811e+09,[솔향기] 홍감자 5Kg (150g이하~90g),0,18000,0,0,NaN,1,0,a56ea55ff34465e2a8f821ee0c56e3e2502760be0099ed...,0,0,home,2021-01-04 00:00:00,2021-01-05 00:00:00,0,3000,0
80183,2699792,d7e14150e95b27de2684eb5d7d15f0d5ffe474f8d2e379...,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,1.575602e+09,[청춘이삼]100% 저온착즙 머루포도즙 50포,0,35000,0,0,NaN,1,0,a97a4cf006543d432e57c5baaf67ebcbbbb31088a25f6d...,0,0,home,2021-01-03 00:00:00,2021-01-04 00:00:00,0,0,0
80184,2699794,8c20d4ce58aa68ad95478671e105837a82940861cf124a...,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,1.570802e+09,햇살바다 고추장참굴비(1kg) 세트,0,65900,0,0,5병 g 200ml,1,0,557c8e0037bad44424120d326b41cc5516e61b4d953afd...,0,0,home,2021-01-04 00:00:00,2021-01-05 00:00:00,0,0,0


In [ ]:
customer_history

,회원번호,상품코드,선호도,장바구니번호,주문번호,회원아이디,상품명,상품 배송비,상품가격,포인트,쿠폰금액,주문옵션,주문수량,옵션가격,업체ID,즉시할인쿠폰,단골할인쿠폰,배송방법,배송일,배송완료예정일,지원가격,배송비,추가배송비
39,60251,1604964672,52.623073,2684151,96164a713b0979d990e00f97a35b514252a50d0b1d8a83...,70d8d65a5a741f4858c7770caacdad92bd9213d3c25c12...,★이웃사촌★ 꿀고구마 5kg,0,21000,0,0,꿀고구마 5kg,1,0,51eab69935667ea62e98f47e491146f7399455ed430395...,0,0,home,0000-00-00,0000-00-00,6300,0,0
28,60251,1604964672,52.623073,2670686,22bc4ac797e309bb7dd05adde6972c1d7fdd752df6dffb...,3ec5da373b219b2c458dba772efe00e20e4de6264b7b50...,★이웃사촌★ 꿀고구마 5kg,0,21000,0,0,꿀고구마 5kg,1,0,51eab69935667ea62e98f47e491146f7399455ed430395...,0,0,home,0000-00-00,0000-00-00,6300,0,0
30,60251,1604964672,52.623073,2673899,8bba3e0ecedaeb90823d93fd407b0c7293a20a15e64cda...,bd535f7650ff3d6cd7cb5a89935e4c4f8aea348467b767...,★이웃사촌★ 꿀고구마 5kg,0,21000,0,0,꿀고구마 5kg,1,0,51eab69935667ea62e98f47e491146f7399455ed430395...,0,0,home,2020-12-03 00:00:00,2020-12-04 00:00:00,6300,0,0
32,60251,1604964672,52.623073,2674696,53bd50f33e1d56af467ade70f39a46dda5a87dfaba8037...,0493fad1c260218f1ceedbeb596ddb791e26dfd5b956b2...,★이웃사촌★ 꿀고구마 5kg,0,21000,0,0,꿀고구마 5kg,1,0,51eab69935667ea62e98f47e491146f7399455ed430395...,0,0,home,2020-12-03 00:00:00,2020-12-04 00:00:00,6300,0,0
33,60251,1604964672,52.623073,2678081,35167da92896b23033b28ce529e7ff22a1356ebf1b070a...,45d8aa48c62db5e91858a5ab4997f0d136386f654f030d...,★이웃사촌★ 꿀고구마 5kg,0,21000,0,0,꿀고구마 5kg,1,0,51eab69935667ea62e98f47e491146f7399455ed430395...,0,0,home,2020-12-03 00:00:00,2020-12-04 00:00:00,6300,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,60251,1600146749,17.054424,2614881,fc27f49caf1f848c086d8dce2b0430630326d9194e2083...,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,［문경몰］오미자 당절임,3000,45000,0,0,오미자 당절임:오미자 당절임(자일로스)6kg,1,4000,60f527abc8a7338fb8cb5452aff1a97e44de28e53278a2...,0,0,home,2020-10-16 00:00:00,2020-10-17 00:00:00,0,0,0
4,60251,1600146749,17.054424,2694831,96c3d899f87a024787dcf236c39cb86e9f31e060c7f71b...,4d03e225edbff9726116847f04cbcd117c7af600b65581...,［문경몰］오미자 당절임,3000,45000,0,0,오미자 당절임:오미자 당절임(자일로스)6kg,1,4000,60f527abc8a7338fb8cb5452aff1a97e44de28e53278a2...,0,0,home,2020-12-21 00:00:00,2020-12-22 00:00:00,0,0,0
3,60251,1600146749,17.054424,2647432,7c95f556188d60563394c41460c73d031747f9045b1ed4...,fa15c7e1d2d05b1cf953c5e94aff5475c662c223d38fff...,［문경몰］오미자 당절임,3000,45000,0,0,오미자 당절임:오미자 당절임(결정과당)10kg,1,50000,60f527abc8a7338fb8cb5452aff1a97e44de28e53278a2...,0,0,home,2020-11-30 00:00:00,2020-12-01 00:00:00,0,0,0
2,60251,1600146749,17.054424,2644089,924862640a244e8c138e37e0b79e73e99e41f2fcca8404...,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,［문경몰］오미자 당절임,3000,45000,0,0,오미자 당절임:오미자 당절임(결정과당)10kg,2,50000,60f527abc8a7338fb8cb5452aff1a97e44de28e53278a2...,0,0,home,2020-11-25 00:00:00,2020-11-26 00:00:00,0,0,0


In [ ]:
recommendations

,장바구니번호,주문번호,회원아이디,상품코드,상품명,상품 배송비,상품가격,포인트,쿠폰금액,주문옵션,주문수량,옵션가격,업체ID,즉시할인쿠폰,단골할인쿠폰,배송방법,배송일,배송완료예정일,지원가격,배송비,추가배송비,60251
0,2087111,b3766513fb393a08da70b368272a97eef2bc3f7f54208a...,7edb360f06acaef2cc80dba16cf563f199d347db4443da...,1.578622e+09,[능금이랑쥬스랑] 청송사과즙 청송사과100%착즙(120ml*60)(구매대행상품),0,21600,0,0,[능금이랑쥬스랑] 청송사과즙 청송사과100%착즙(120ml*60)(구매대행상품),43,0,42d149219525ad9a6d6475f4ea95138dba7984566452ab...,0,0,home,2020-01-16 00:00:00,2020-01-16 00:00:00,0,0,0,0.071302
1,2087115,550ec01a5ec445a8f16b5602c54f9c1537202797d3f8e5...,7edb360f06acaef2cc80dba16cf563f199d347db4443da...,1.578623e+09,[능금이랑쥬스랑] 청송사과즙 청송사과100%착즙(120ml*30)(구매대행상품),0,10800,0,0,[능금이랑쥬스랑] 청송사과즙 청송사과100%착즙(120ml*30)(구매대행상품),48,0,42d149219525ad9a6d6475f4ea95138dba7984566452ab...,0,0,home,2020-01-16 00:00:00,2020-01-16 00:00:00,0,0,0,0.071289
2,2088937,bdbf02fac876b0f5c29c48bf4b38ac3593906e2d34515a...,a8e885e8b77a447052f83caa93f04ddb3be1f5ade2832b...,1.577949e+09,청송 달달한사과 선물세트 부사 5kg[15~16과],0,40000,0,0,청송 달달한사과 선물세트 부사 5kg[15~16과],1,0,a139a934879db1cca204d7175794423cffeaa6e10cc3b1...,6000,0,home,2020-01-15 00:00:00,2020-01-16 00:00:00,0,0,0,0.071255
3,2089114,8348411f27bb8091a18bd5daa64ba24835a9d79e31d549...,3db9cda6720d9ea9be53a096f0caa170ceccd9cc1b7201...,1.577870e+09,[농업회사법인 햇살찬山사과] 껍질째먹는봉화사과5kg 설선물용(대과15과),0,60000,0,5000,[농업회사법인 햇살찬山사과] 껍질째먹는봉화사과5kg 설선물용(대과15과),1,0,0762978285ae4c072cf34a2f893e68f6dba875213eccd6...,20000,0,home,2020-01-14 00:00:00,2020-01-15 00:00:00,0,0,0,0.071255
4,2128141,c575c377dd7bf875475b3da8c5f4d57c6f1e028423ff44...,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,1.577870e+09,[농업회사법인 햇살찬山사과] 껍질째먹는봉화사과5kg 설선물용(대과15과),0,60000,0,0,NaN,1,0,0762978285ae4c072cf34a2f893e68f6dba875213eccd6...,0,0,home,2020-04-09 00:00:00,2020-04-10 00:00:00,0,0,0,0.071255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23783,2698829,b5ff90d5876bd88d17ff3729b90f0722faac5153c4da4d...,da5e11efa36720a4211ac89acf1479952e99b35636f006...,1.608101e+09,(문경약돌돼지) 문경약돌돼지 목살(냉동) 500g*2,3000,15800,0,0,(문경약돌돼지) 문경약돌돼지 목살(냉동) 500g*2,4,0,e9c31c99ca65c3d512e7a8b9e0a995c82cd52064b695c9...,0,0,home,2020-12-30 00:00:00,2020-12-31 00:00:00,0,0,0,0.071254
23784,2699129,dafea383ff1a0b4242fc53bb79a34aa52b0a8d3975865f...,80269b136c070dcb9597d6bfdf437d6a1be28525bb2143...,1.608101e+09,(문경약돌돼지) 문경약돌돼지 목살(냉동) 500g*2,3000,15800,0,0,(문경약돌돼지) 문경약돌돼지 목살(냉동) 500g*2,1,0,e9c31c99ca65c3d512e7a8b9e0a995c82cd52064b695c9...,0,0,home,2020-12-30 00:00:00,2020-12-31 00:00:00,0,0,0,0.071254
23785,2698936,8b9a34e0bb92857e0c4ace661147133916d3f57c1fdc3e...,2e53ec665352f3f667d4dcf5442a68e0ddf99c545c1dfe...,1.608102e+09,(문경약돌돼지) 문경약돌돼지 LA식 갈비(냉동) 1kg,3000,13800,0,0,(문경약돌돼지) 문경약돌돼지 LA식 갈비(냉동) 1kg,1,0,e9c31c99ca65c3d512e7a8b9e0a995c82cd52064b695c9...,0,0,home,2020-12-30 00:00:00,2020-12-31 00:00:00,0,0,0,0.071262
23786,2699533,baf42f54e04a650e74b2388c0457318740ef671691ed2a...,5d27900cb46a3bb72bc5f29a488e213d433fd074e92244...,1.606737e+09,예천두레농장 생알땅콩2kg,0,35000,0,0,예천두레농장 생알땅콩2kg,1,0,b33e237445a11431ed71161eccee6145eac3df3a63c7a0...,0,0,home,2021-01-04 00:00:00,2021-01-05 00:00:00,0,0,0,0.071255


In [ ]:
sorted_customer_pred.transpose().reset_index()["상품코드"]

0       1592373538
1       1595904905
2       1589126164
3       1588214366
4       1593673443
           ...    
1585    1603865613
1586    1604385982
1587    1584201031
1588    1606464566
1589    1582691001
Name: 상품코드, Length: 1590, dtype: int64

In [ ]:
df_svd_preds.reset_index(inplace=True)

In [ ]:
df_svd_preds["회원번호"]

0            1
1        60251
2        60596
3        60603
4        60729
         ...  
8983    169279
8984    169280
8985    169282
8986    169286
8987    169295
Name: 회원번호, Length: 8988, dtype: int64

In [ ]:
# 회원번호 1인 고객이 행렬 분해 기반의 추천 시스템으로부터 받은 추천 상품 목록
num_recommendations = 5
customer_id = 60729


# 고객 구매이력, 선호도
already_rated,prediction= recommend_merchandise(df_svd_preds, customer_id, df_merchandise, df_ratings, num_recommendations)

In [ ]:
already_rated

,회원번호,상품코드,선호도,장바구니번호,주문번호,회원아이디,상품명,상품 배송비,상품가격,포인트,쿠폰금액,주문옵션,주문수량,옵션가격,업체ID,즉시할인쿠폰,단골할인쿠폰,배송방법,배송일,배송완료예정일,지원가격,배송비,추가배송비
0,60729,1598427460,57.537618,2572015,781dba46a06dcd9cf64f504feb67ab8d70592409bfc4ee...,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,[아름드리사과] 2020 햇 사과_ 가정용 홍로사과 5kg 소과 20-23과내,0,20900,0,0,[아름드리사과] 2020 햇 사과_ 가정용 홍로사과 5kg 소과 20-23과내,1,0,14cd5fb3bdf604753040ecf9c3e9165bccbf6ed1db22d1...,0,0,home,2020-08-31 00:00:00,2020-09-01 00:00:00,0,0,0
37,60729,1598427460,57.537618,2584277,780455f716faa4f7a9cc6968eb885c9f284393fc4de723...,a1dff369905e1706147ced6fe409f19f0fab899ce15f07...,[아름드리사과] 2020 햇 사과_ 가정용 홍로사과 5kg 소과 20-23과내,0,21900,0,0,[아름드리사과] 2020 햇 사과_ 가정용 홍로사과 5kg 소과 20-23과내,1,0,14cd5fb3bdf604753040ecf9c3e9165bccbf6ed1db22d1...,0,0,home,2020-09-08 00:00:00,2020-09-09 00:00:00,0,0,0
53,60729,1598427460,57.537618,2608039,c89b76d3a4eebf5733e39a966f5e75864caaa2c042f2a8...,42fbbf9ddf77133f0d03bd2f3c5b1a82428f3e202d7234...,[아름드리사과] 2020 햇 사과_ 가정용흠집 빨간사과 5kg 소과 20-23과내,0,22900,0,0,[아름드리사과] 2020 햇 사과_ 가정용흠집 빨간사과 5kg 소과 20-23과내,1,0,14cd5fb3bdf604753040ecf9c3e9165bccbf6ed1db22d1...,0,0,home,2020-10-05 00:00:00,2020-10-06 00:00:00,0,0,0
52,60729,1598427460,57.537618,2606478,628363f9564c4909cf4c876400ef14b19efa1193f042ab...,d4c8130af67911ead52fd21d10da28c922aca11a63a08f...,[아름드리사과] 2020 햇 사과_ 가정용흠집 빨간사과 5kg 소과 20-23과내,0,22900,0,0,[아름드리사과] 2020 햇 사과_ 가정용흠집 빨간사과 5kg 소과 20-23과내,1,0,14cd5fb3bdf604753040ecf9c3e9165bccbf6ed1db22d1...,0,0,home,2020-09-25 00:00:00,2020-09-26 00:00:00,0,0,0
51,60729,1598427460,57.537618,2606477,4f5f545e0ea5420ad59cceaff0768cd026845d72e2c4ba...,d4c8130af67911ead52fd21d10da28c922aca11a63a08f...,[아름드리사과] 2020 햇 사과_ 가정용흠집 빨간사과 5kg 소과 20-23과내,0,22900,0,0,[아름드리사과] 2020 햇 사과_ 가정용흠집 빨간사과 5kg 소과 20-23과내,1,0,14cd5fb3bdf604753040ecf9c3e9165bccbf6ed1db22d1...,0,0,home,2020-09-25 00:00:00,2020-09-26 00:00:00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,60729,1602486335,19.340911,2640367,fb0669f5a8dc5edd6b10cf734cfb4e443b1e0d84376108...,7d5e61962435b1e025f042bed82d974f9fd11af8bd461e...,★이웃사촌★ [내서서리골마을]오아시스 건오미자400g,0,25500,0,0,[내서서리골마을]오아시스 건오미자400g,1,0,5f90d34c68fb01f90df340c684d12fdb25346c06af0853...,0,0,home,2020-11-23 00:00:00,2020-11-24 00:00:00,5040,0,0
77,60729,1602486335,19.340911,2640556,0299918cc207dac56a79c83bd0242c76f009ccd58175bd...,505d017f065cdb01bc221bc6f45e892c032c0e082342ad...,★이웃사촌★ [내서서리골마을]오아시스 건오미자400g,0,25500,0,0,[내서서리골마을]오아시스 건오미자400g,1,0,5f90d34c68fb01f90df340c684d12fdb25346c06af0853...,0,0,home,2020-11-23 00:00:00,2020-11-25 00:00:00,5040,0,0
78,60729,1602486335,19.340911,2642271,b0240a9e4c3bb085e3590329b5a85ddd5db02fffd907e9...,a1a7a0386570ba990f16d496e2907bf992c7dc7c313707...,★이웃사촌★ [내서서리골마을]오아시스 건오미자400g,0,25500,0,0,[내서서리골마을]오아시스 건오미자400g,1,0,5f90d34c68fb01f90df340c684d12fdb25346c06af0853...,0,0,home,2020-11-23 00:00:00,2020-11-25 00:00:00,5040,0,0
79,60729,1602486335,19.340911,2672407,9c95b5390c6cafc70a2534852d5dc1d53265b1a6b9c9de...,9e7c7cb28526442f26f8b09a3d221a1753ba99840ae4c7...,★이웃사촌★ [내서서리골마을]오아시스 건오미자400g,0,25500,0,0,[내서서리골마을]오아시스 건오미자400g,1,0,5f90d34c68fb01f90df340c684d12fdb25346c06af0853...,0,0,home,2020-12-02 00:00:00,2020-12-03 00:00:00,7650,0,0
